In [3]:
import math
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

# Data

In [4]:
import numpy as np
import os, sys
import random #shuffle
from netCDF4 import Dataset
def CMIPdata(Xdata, Ydata, out , myform):
    inp1 = Dataset(Xdata,'r')
    inp2 = Dataset(Ydata,'r')
    sst_1 = np.zeros((myform,140,12,24,72))

    #首年序列
    #i= 21个模式
    for i in range(myform):
        sst_1[i,:,:,:,:] = inp1.variables['sst1'][1+141*i:141+141*i,0:12,:,:]
    #(21,140,12,24,72)
    #首年序列

    
    #flatted扁平化                                             
    sst_2 = np.zeros((myform,1680,24,72))
    for i in range(myform):
        for j in range(140):
            sst_2[i,j*12:(j+1)*12,:,:] = sst_1[i,j,:,:,:]
    
        #(21,1680,24,72)
        #每个模式 139*12+1 = 1680 个序列 序列长度为12

    #丢弃1个月 1862.1
    sst_2 = sst_2[:,1:,:,:]
    #(form,1679,24,72)


    winsize = 12
    #winnum = 1679-winsize +1 #1668
    """跳跃六个月滑窗 = 6"""
    ts=6 
    #(1679-12)/6+1 =27
    winnum=int((1679-12)/ts +1) 
    
    sst_3 = np.zeros((myform,winnum,12,24,72))
    for i in range(myform):
        for j in range(winnum):
            #print(j)
            sst_3[i,j,:,:,:] = sst_2[i,(j*ts):(j*ts)+winsize,:,:]
            

    sst_4 = np.zeros((int(winnum*myform),12,24,72))
    #t300_4 = np.zeros((int(winnum*myform),12,24,72))
    for i in range(myform):
        sst_4[i*winnum:(i+1)*winnum,:,:,:] = sst_3[i,:,:,:,:]


    #sst_4 = sst_4.swapaxes(1, 3)
    trX = sst_4
    del sst_1,sst_2,sst_3,sst_4
    #trX = trX.reshape(int(winnum*myform),12,1728)

    #保存np数组
    #(5859, 12, 24, 72, 2)
    #np.save("./CMIPdata/CMIP_trX_21_ts6_out.npy",trX) 

    #label
    #平铺
    pr_1 = np.zeros(((myform),1692))#21,1692
    for i in range(myform):
        for j in range(141):
            pr_1[i,j*12:(j+1)*12] = inp2.variables['pr'][(141*i)+j,:,0,0]
            
    #out
    pr_2 =  np.zeros((myform,winnum,out))#form,1668,1
    for i in range(myform):
        for j in range(winnum):
            pr_2[i,j,:] = pr_1[i,(j*ts):(j*ts)+out]

    trY = np.zeros((int(winnum*myform),out))#form*1668,1
    for i in range(myform):
        trY[i*winnum:(i+1)*winnum,:] = pr_2[i,:,:]

    #np.save("./CMIPdata/CMIP_trY_21_ts6_out%s.npy"%out,trY)
    trY_decoder_input = np.zeros((int(winnum*myform),out))
    trY_decoder_input[:,1:] = trY[:,:-1]

    #return trX[1:,:,:] ,trY.reshape(-1,24,1)[1:,:,:] ,trY_decoder_input
    return trX ,trY.reshape(-1,24,1) ,trY_decoder_input.reshape(-1,24,1)

def GOSDAdata(Xdata, Ydata, out):
    #test data
    inp11 = Dataset(Xdata,'r')
    inp22 = Dataset(Ydata,'r')
                                              
    sst_11 = np.zeros((33,12,24,72)) #1983-2015
    #t300_11 = np.zeros((33,12,24,72))

    sst_11[:,:,:,:] = inp11.variables['sst'][3:,0:12,:,:]
    #t300_11[:,:,:,:] = inp11.variables['t300'][3:,0:12,:,:]
    #(33,12,24,72)

    sst_22 = np.zeros((396,24,72))
    #t300_22 = np.zeros((396,24,72))

    for i in range(33):
        sst_22[i*12:(i+1)*12,:,:] = sst_11[i,:,:,:]
        #t300_22[i*12:(i+1)*12,:,:] = t300_11[i,:,:,:]
    #(396,24,72)
    #丢弃一个月，便于制作滑窗序列
    sst_22 = sst_22[1:,:,:]
    #t300_22 = t300_22[1:,:,:]#(395,24,72)

    #滑窗
    winsize = 12
    winnum = 395-winsize + 1 #384
    sst_33 = np.zeros((winnum, winsize, 24, 72))#(384, 12, 24, 72)
    #t300_33 = np.zeros((winnum, winsize, 24, 72))

    for i in range(winnum):
        sst_33[i,:,:,:] = sst_22[i:i+12,:,:]
        #t300_33[i,:,:,:] = t300_22[i:i+12,:,:]
    #(384,12,24,72)


    #channel = 2
    #testX = np.zeros((winnum,12,24,72,2))#384
    #testX[:,:,:,:,0] = sst_33
    #testX[:,:,:,:,1] = t300_33
    #sst_33 = sst_33.swapaxes(1, 3)

    testX = sst_33#384
    #testX[:,:,:,1] = t300_33.reshape(384,12,1728)
    del sst_11,sst_22,sst_33
    #del t300_11,t300_22,t300_33
    #testX = testX.reshape(384,12,3456)

    #label
    inpv22 = np.zeros((408))#2017-1984 +1=34
    for i in range(34):
        inpv22[i*12:(i+1)*12] = inp22.variables['pr'][i+2,:,0,0]# +2:从1984开始
        #(408)

    testY = np.zeros((winnum,out,1))
    #out = 1
    #滑窗
    for i in range(winnum):
        testY[i,:,0] = inpv22[i:i+out]#24
    #(384,out,1)

    return testX, testY

def SODAdata(Xdata, Ydata, out):
    inp1 = Dataset(Xdata,'r')
    inp2 = Dataset(Ydata,'r')

    #time_step = 1 month                                               
    sst_1 = np.zeros((99,12,24,72))
    #t300_1 = np.zeros((99,12,24,72))

    sst_1[:,:,:,:] = inp1.variables['sst'][1:,0:12,:,:]
    #sst_1[:,:,:,:] = inp1.variables['t300'][1:,0:12,:,:]
    #(99,12,24,72)
    #1872-1970

    sst_2 = np.zeros((1188,24,72))
    #t300_2 = np.zeros((1188,24,72))

    for i in range(99):
        sst_2[i*12:(i+1)*12,:,:] = sst_1[i,:,:,:]
        #t300_2[i*12:(i+1)*12,:,:] = t300_1[i,:,:,:]
    #(1188,24,72)
    #丢弃一个月，便于制作滑窗序列
    sst_2 = sst_2[1:,:,:]
    #t300_2 = t300_2[1:,:,:]#(1187,24,72)

    sst_3 = np.zeros((1176,12,24,72))
    #t300_3 = np.zeros((1176,12,24,72))
    #滑窗
    for i in range(1176):
        sst_3[i:,:,:] = sst_2[i:i+12,:,:]
        #t300_3[i,:,:,:] = sst_2[i:i+12,:,:]
    #(1176,12,24,72)
    trX = sst_3

    #channel = 2
    #trX = np.zeros((1176,12,24,72,2))
    #trX[:,:,:,:,0] = sst_3
    #trX[:,:,:,:,1] = t300_3

    #保存np数组
    #np.save("./SODAdata/SODA_trX_ts1_out.npy",trX)

    #label
    inpv2 = np.zeros((1200))
    for i in range(100):
        inpv2[i*12:(i+1)*12] = inp2.variables['pr'][i,:,0,0]
    #(1200)

    #out = 1
    trY = np.zeros((1176,out,1))
    #滑窗
    for i in range(1176):
        trY[i,:,0] = inpv2[i:i+out]#24
    #(1176,24)
    
    
    trY_decoder_input = np.zeros((1176,out,1))
    trY_decoder_input[:,1:,:] = trY[:,:-1,:]

    #np.save("./SODAdata/SODA_trY_ts1_out%s.npy"%out,trY) 
    return trX , trY, trY_decoder_input

trX, trY, trY_decoder_input = CMIPdata('/home/d/Q/liyanqiu/saconvlstm/CMIP5.input.36mon.1861_2001.nc',\
                                          '/home/d/Q/liyanqiu/saconvlstm/CMIP5.label.12mon.1863_2003.nc', 24,  21)

testX, testY = GOSDAdata('/home/d/Q/liyanqiu/saconvlstm/GODAS.input.36mon.1980_2015.nc',\
                         '/home/d/Q/liyanqiu/saconvlstm/GODAS.label.12mon.1982_2017.nc', 24)

trX_SODA, trY_SODA ,trY_decoder_input_SODA = SODAdata('/home/d/Q/liyanqiu/saconvlstm/SODA.input.36mon.1871_1970.nc',
                              '/home/d/Q/liyanqiu/saconvlstm/SODA.label.12mon.1873_1972.nc',24)

# resize 72*72
trX.resize((trX.shape[0], trX.shape[1], 72, 72))
trX_SODA.resize((trX_SODA.shape[0], trX_SODA.shape[1], 72, 72))
testX.resize((testX.shape[0], testX.shape[1], 72, 72))

indices = np.arange(trX.shape[0])
np.random.shuffle(indices)
trX = trX[indices]
trY = trY[indices]
trY_decoder_input = trY_decoder_input[indices]


# shuffle
indices_soda = np.arange(trX_SODA.shape[0])
np.random.shuffle(indices_soda)
trX_SODA = trX_SODA[indices_soda]
trY_SODA = trY_SODA[indices_soda]
trY_decoder_input_SODA = trY_decoder_input_SODA[indices_soda]


class MyDataSet(Data.Dataset):
    """自定义DataLoader"""
    def __init__(self, enc_inputs,  dec_outputs):
        super(MyDataSet, self).__init__()
        self.enc_inputs = enc_inputs
        #self.dec_inputs = dec_inputs
        self.dec_outputs = dec_outputs

    def __len__(self):
        return self.enc_inputs.shape[0]

    def __getitem__(self, idx):
        return self.enc_inputs[idx],  self.dec_outputs[idx]
    
batch_size = 16
trainsize = 2500 #选择cmip训练数量
trX = trX[:trainsize]
trY = trY[:trainsize]



# expand channel
trX = np.expand_dims(trX, axis=2)
trX_SODA = np.expand_dims(trX_SODA, axis=2)
testX = np.expand_dims(testX, axis=2)

#train split
split_index = int(trX.shape[0]*0.8)
split_index_soda = int(trX_SODA.shape[0]*0.8)

print("split_index_soda:" + str(split_index_soda))
print("split_index:" + str(split_index))


target_month = 1
loader_train = Data.DataLoader(
    MyDataSet(
        trX[:split_index].astype(np.float32), 
        trY[:split_index, target_month-1:target_month].astype(np.float32).reshape((-1,1))), 
    batch_size, shuffle = True)
loader_valid = Data.DataLoader(
    MyDataSet(
        trX[split_index:].astype(np.float32), 
        trY[split_index:, target_month-1:target_month].astype(np.float32).reshape((-1,1))), 
    batch_size, shuffle=False)

loader_train_sado = Data.DataLoader(
    MyDataSet(
        trX_SODA[:split_index_soda].astype(np.float32), 
        trY_SODA[:split_index_soda, target_month-1:target_month].astype(np.float32).reshape((-1,1))), 
    batch_size, shuffle = True)
loader_valid_soda = Data.DataLoader(
    MyDataSet(
        trX_SODA[split_index_soda:].astype(np.float32), 
        trY_SODA[split_index_soda:, target_month-1:target_month].astype(np.float32).reshape((-1,1))), 
    batch_size, shuffle=False)

#testdataloader

loader_test = Data.DataLoader(
    MyDataSet(
        testX.astype(np.float32), 
        testY[:, target_month-1:target_month].astype(np.float32).reshape((-1,1))), 
    batch_size, shuffle = False)

testY = testY[:, target_month-1:target_month].astype(np.float32).reshape((-1,1))

print(trX.shape)
print(trX_SODA .shape)
print(testY.shape)
print(len(testX))
print(trX_SODA.shape)
print("----------------------------Train Data----------------------------")
print(trX[:split_index].astype(np.float32).shape)
print(trY[:split_index, target_month-1:target_month].reshape((-1,1)).shape)


split_index_soda:940
split_index:2000
(2500, 12, 1, 72, 72)
(1176, 12, 1, 72, 72)
(384, 1)
384
(1176, 12, 1, 72, 72)
----------------------------Train Data----------------------------
(2000, 12, 1, 72, 72)
(2000, 1)


# Model

In [5]:
from torch import nn, einsum
import torch.nn.functional as F
from einops import rearrange, repeat


class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.fn = fn
        self.norm = nn.LayerNorm(dim)

    def forward(self, x, *args, **kwargs):
        x = self.norm(x)
        return self.fn(x, *args, **kwargs)
    
#feed forward
class GEGLU(nn.Module):
    def forward(self, x):
        x, gates = x.chunk(2, dim = -1) # 在给定的维度上将张量进行分块。沿着-1分成两块
        return x * F.gelu(gates)

class FeedForward(nn.Module):
    def __init__(self, dim, mult = 4, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, dim * mult * 2),
            GEGLU(),
            nn.Dropout(dropout),
            nn.Linear(dim * mult, dim)#geglu大小除以2
        )

    def forward(self, x):
        return self.net(x)
    
#attention
def attn(q, k, v):
    sim = einsum('b i d, b j d -> b i j', q, k)
    attn = sim.softmax(dim = -1)
    out = einsum('b i j, b j d -> b i d', attn, v)
    return out

class Attention(nn.Module):
    def __init__(
        self,
        dim,
        dim_head = 64,
        heads = 8,
        dropout = 0.
    ):
        super().__init__()
        self.heads = heads
        self.scale = dim_head ** -0.5
        inner_dim = dim_head * heads

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)
        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x, einops_from, einops_to, **einops_dims):
        h = self.heads
        q, k, v = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> (b h) n d', h = h), (q, k, v))

        q *= self.scale

        # splice out classification token at index 1
        (cls_q, q_), (cls_k, k_), (cls_v, v_) = map(lambda t: (t[:, 0:1], t[:, 1:]), (q, k, v))

        # let classification token attend to key / values of all patches across time and space
        cls_out = attn(cls_q, k, v)

        # rearrange across time or space
        q_, k_, v_ = map(lambda t: rearrange(t, f'{einops_from} -> {einops_to}', **einops_dims), (q_, k_, v_))

        # expand cls token keys and values across time or space and concat
        r = q_.shape[0] // cls_k.shape[0]
        cls_k, cls_v = map(lambda t: repeat(t, 'b () d -> (b r) () d', r = r), (cls_k, cls_v))

        k_ = torch.cat((cls_k, k_), dim = 1)
        v_ = torch.cat((cls_v, v_), dim = 1)

        # attention
        out = attn(q_, k_, v_)

        # merge back time or space 合并时间或空间
        out = rearrange(out, f'{einops_to} -> {einops_from}', **einops_dims)

        # concat back the cls token 连接回来的cls token
        out = torch.cat((cls_out, out), dim = 1)

        # merge back the heads
        out = rearrange(out, '(b h) n d -> b n (h d)', h = h)

        # combine heads out 合并head输出
        return self.to_out(out) # batch * dim
    
    
# main classes
class TimeSformer(nn.Module):
    def __init__(
        self,
        *,
        dim,
        num_frames,
        num_classes,
        image_size = 72,
        patch_size = 12,
        channels = 1,
        depth = 12,
        heads = 8,
        dim_head = 64,
        attn_dropout = 0.,
        ff_dropout = 0.
    ):
        super().__init__()
        #符合则往下运行
        assert image_size % patch_size == 0, 'Image dimensions must be divisible by the patch size.'

        num_patches = (image_size // patch_size) ** 2 #平方个数 //向下取整
        num_positions = num_frames * num_patches # 12*num_patches
        patch_dim = channels * patch_size ** 2 # patch大小

        self.patch_size = patch_size
        self.to_patch_embedding = nn.Linear(patch_dim, dim)
        self.pos_emb = nn.Embedding(num_positions + 1, dim)
        self.cls_token = nn.Parameter(torch.randn(1, dim))# [1, dim]

        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, dim_head = dim_head, heads = heads, dropout = attn_dropout)), # Time attention
                PreNorm(dim, Attention(dim, dim_head = dim_head, heads = heads, dropout = attn_dropout)), # Spatial attention
                PreNorm(dim, FeedForward(dim, dropout = ff_dropout)) # Feed Forward
            ]))

        
        self.to_out = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )
    
        #self.to_out = nn.Linear(dim, num_classes)

    def forward(self, video):
        b, f, _, h, w, *_, device, p = *video.shape, video.device, self.patch_size
        assert h % p == 0 and w % p == 0, f'height {h} and width {w} of video must be divisible by the patch size {p}'

        n = (h // p) * (w // p)
        video = rearrange(video, 'b f c (h p1) (w p2) -> b (f h w) (p1 p2 c)', p1 = p, p2 = p)
        
        tokens = self.to_patch_embedding(video)

        cls_token = repeat(self.cls_token, 'n d -> b n d', b = b)# [batch, 1, dim]
        x =  torch.cat((cls_token, tokens), dim = 1)
        x += self.pos_emb(torch.arange(x.shape[1], device = device))

        for (time_attn, spatial_attn, ff) in self.layers:
            x = time_attn(x, 'b (f n) d', '(b n) f d', n = n) + x
            x = spatial_attn(x, 'b (f n) d', '(b f) n d', f = f) + x
            x = ff(x) + x

        cls_token = x[:, 0]
        
        return self.to_out(cls_token)
    
    
#Parameters
device = 'cuda'
DIM = 128
IMAGE_SIZE = 72
PATCH_SIZE = 12 #12 24 8
NUM_CLASSES = 1
NUM_FRAMES = 12
DEPTH = 1 # block个数
HEADS = 8
DIM_HEAD = 64
ATTN_DROPOUT = 0.1
FF_DROPOUT = 0.1
#ITERATIONS = 20# epoch


# (batch x frames x channels x height x width)
model = TimeSformer(dim = DIM, image_size = IMAGE_SIZE, patch_size = PATCH_SIZE, num_frames = NUM_FRAMES, \
        num_classes = NUM_CLASSES, depth = DEPTH, heads = HEADS, dim_head = DIM_HEAD, attn_dropout = ATTN_DROPOUT, \
            ff_dropout = FF_DROPOUT).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())
print("if on cuda:",next(model.parameters()).is_cuda)

if on cuda: True


# Train

In [6]:
import pandas as pd 
import datetime
dfhistory = pd.DataFrame(columns = ["epoch","loss","val_loss"]) 
print("Start Training...")
nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print("=========="*8 + "%s"%nowtime)
bestloss = float("inf")
epoch = 20
for epoch in range(1,epoch+1):
    #train----------------------------------------------------
  model.train()
  loss_sum = 0.0
  step = 1
  for step,(enc_inputs, dec_outputs) in enumerate(loader_train, 1):
      optimizer.zero_grad()

      # 正向
      enc_inputs, dec_outputs = enc_inputs.to(device), dec_outputs.to(device)
      y_pred = model(enc_inputs)
      
      loss = criterion(y_pred, dec_outputs) 

      # 反向
      loss.backward()
      optimizer.step()

      # print
      loss_sum += loss.item()
      #if step%40 == 0:
        #print(("[step = %d] loss: %.3f, ") % (step, loss_sum/step))


  #valid
  model.eval()
  val_loss_sum = 0.0
  val_step = 1

  for val_step, (enc_inputs, dec_outputs) in enumerate(loader_valid, 1):
    with torch.no_grad():# 节点不进行求梯度
      enc_inputs, dec_outputs = enc_inputs.to(device), dec_outputs.to(device)
      outputs = model(enc_inputs)
      #val_loss = criterion(outputs.view(-1), dec_outputs.view(-1)) 
      val_loss =criterion(outputs, dec_outputs) 

    val_loss_sum += val_loss.item()


  # log
  info = (epoch, loss_sum/step, val_loss_sum/val_step)
  dfhistory.loc[epoch-1] = info # epoch从1开始 index -1

  # print
  print(("\nEPOCH = %d, loss = %.3f,"+" val_loss = %.3f, ")% info)
  nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
  print("\n"+"=========="*8 + "%s"%nowtime)
  
  if val_loss_sum <= bestloss: #换成小于等于比小于效果更好
        bestloss = val_loss_sum
        bestmodel = model
        

print('Finished Training...')

Start Training...
================================================================================2022-03-10 20:46:50

EPOCH = 1, loss = 0.693, val_loss = 0.624, 

================================================================================2022-03-10 20:46:54

EPOCH = 2, loss = 0.678, val_loss = 0.630, 

================================================================================2022-03-10 20:46:57

EPOCH = 3, loss = 0.667, val_loss = 0.672, 

================================================================================2022-03-10 20:46:59

EPOCH = 4, loss = 0.671, val_loss = 0.622, 

================================================================================2022-03-10 20:47:01

EPOCH = 5, loss = 0.674, val_loss = 0.650, 

================================================================================2022-03-10 20:47:03

EPOCH = 6, loss = 0.665, val_loss = 0.628, 

================================================================================2022-03-10 20:47:06

EPOCH

# Evaluation

In [5]:
out = []
# batch_size
for step,(inputs, _) in enumerate(loader_test, 1):
    pred = model(inputs.to(device))
    out.append(pred.squeeze().cpu().detach().numpy())

out = np.array(out)    
out.shape

#——————————————cor————————————————
cor = np.zeros((1))
cor = np.corrcoef(testY[:,0],out.reshape((-1)))[0,1]
print(cor)


0.04751934806165872


In [ ]:
pre = model(torch.from_numpy(testX[:10].astype(np.float32)).to(device))
pre.cpu().detach().numpy()
pre.shape

In [6]:
a = np.array([[0,1,2], [3,4,5]])
a.resize(3, 3)
print(a)


[[0 1 2]
 [3 4 5]
 [0 0 0]]


In [41]:
a = np.array([[0,1,2], [3,4,5]])
a.reshape((-1))

array([0, 1, 2, 3, 4, 5])